In [1]:
from hl_navigation import Holonomic, Disc, Twist2, Pose2, Kinematic, Neighbor, Disc, LineSegment
from hl_navigation._hl_navigation import Behavior, GeometricState

In [2]:
import numpy as np

# Jerome: what about the radius and safety margin?   

Vector2 = np.ndarray

class SocialForceBehavior(Behavior, GeometricState):
    def __init__(self, kinematic: Kinematic, radius: float, tau: float = 0.5, 
                 step_duration: float = 1.0,  phi: float = 1.0):
        Behavior.__init__(self, kinematic, radius)
        GeometricState.__init__(self)
        self.tau = tau
        self.step_duration = step_duration
        self.cos_phi = np.cos(phi)

    def neighbor_repulsive_potential_gradient(self, b) -> Vector2:
        # gradient of monotonic decreasing function V(b)
        return np.array((1.0, 1.0))

    def obstacle_repulsive_potential_gradient(self, distance) -> Vector2:
        # gradient of monotonic decreasing function U(distance)
        return np.array((1.0, 1.0))
        
    def neighbor_repulsion_force(self, neighbor: Neighbor) -> Vector2:
        delta = neighbor.position - self.position
        step = neighbor.velocity * self.step_duration
        b = 0.5 * np.sqrt(np.linalg.norm(delta) + np.linalg.norm(delta - step) - 
                          np.linalg.norm(step) ** 2)
        return -self.neighbor_repulsive_potential_gradient(b)

    def disc_repulsion_force(self, obstacle: Disc) -> Vector2:      
        distance = np.linalg.norm(obstacle.position - self.position)
        return -self.obstacle_repulsive_potential_gradient(distance)

    def line_repulsion_force(self, line: LineSegment) -> Vector2:
        distance = line.distance_from_point(self.position)
        return -self.obstacle_repulsive_potential_gradient(distance)    
    
    def effective_sight(self, e: Vector2, force: Vector2, sign: int) -> float:
        if np.dot(e, sign * force) > self.cos_phi * np.linalg.norm(force):
            return force
        return np.zeros(2)

    # TODO attractive effects
    # TODO fluctuations
    def compute_desired_velocity(self, dt: float) -> Vector2:
        # Target is in general an area, 
        # then target_position is the closed point in that area
        e = self.target_position - self.position
        target_velocity = e * self.optimal_speed
        # acceleration towards desired velocity        
        force = (target_velocity - self.velocity) / self.tau
        # repulsion from neighbors
        force += sum(
            self.effective_sight(e, self.neighbor_repulsion_force(neighbor), -1) 
            for neighbor in self.neighbors)
        force += sum(
            self.effective_sight(e, self.disc_repulsion_force(obstacle), 1)
            for obstacle in self.static_obstacles)
        force += sum(
            self.effective_sight(e, self.line_repulsion_force(obstacle), 1)
            for obstacle in self.line_obstacles)
        desired_velocity = behavior.actuated_twist.velocity + dt * force
        # no need to clamp norm ... this will be done the superclass
        return desired_velocity        

In [3]:
# behavior_names() -> get_behavior_names() or use a value

In [4]:
behavior = SocialForceBehavior(Holonomic(1.0, 1.0), 0.1)

In [5]:
behavior.horizon = 5.0
behavior.position = (0.0, 0.00)
behavior.target_position = (10.0, 0.0)
behavior.optimal_speed = 0.5
behavior.static_obstacles = [Disc(position=(1.5, 0.0), radius=0.5)]
behavior.neighbor_obstacles = [Disc(position=(1.5, 0.0), radius=0.5)]
behavior.cmd_twist(0.1, True)


Twist2((1.000000, 0.000000), 0.000000, True)

In [6]:
behavior.compute_desired_velocity(0.1)

array([2., 0.], dtype=float32)